In [10]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping



In [11]:
from Data_Preparation.fish_survival_data_preparation import create_fish_pipeline, prepare_fish_data


In [12]:
# Loading and Preprocessing the Data here. Basically just getting the data prepared for 
# DNN model.
X_train, X_dev, X_test, y_train_orig, y_dev_orig, y_test_orig = prepare_fish_data(ratios=(0.1, 0.1))

pipeline = create_fish_pipeline()
X_train_scaled = pipeline.fit_transform(X_train)
X_dev_scaled = pipeline.transform(X_dev)
X_test_scaled = pipeline.transform(X_test)

# Normalizing the target value here as the variance is very low in y.

target_scaler = StandardScaler()
y_train_scaled = target_scaler.fit_transform(y_train_orig.to_numpy().reshape(-1, 1)).flatten()
y_dev_scaled = target_scaler.transform(y_dev_orig.to_numpy().reshape(-1, 1)).flatten()
y_test_scaled = target_scaler.transform(y_test_orig.to_numpy().reshape(-1, 1)).flatten()


In [13]:
# Tried a model with more layers but got not so good reusults (we can alomst say bad) so 
# Buildt a new DNN Model function with less layers.

def build_dnn_model(input_dim):
    model = Sequential([
        Input(shape=(input_dim,)),
        Dense(128, activation='relu'),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dropout(0.2),
        Dense(1, activation='relu') 
    ])
    return model

In [14]:
model = build_dnn_model(X_train_scaled.shape[1])
model.compile(optimizer=Adam(), loss='mse')

In [ ]:
# Training the Model here
history = model.fit(
    X_train_scaled, y_train_scaled,
    validation_data=(X_dev_scaled, y_dev_scaled),
    epochs=100,
    batch_size=32,
    verbose=1,
    callbacks=[EarlyStopping(patience=100, restore_best_weights=True)]
)

Epoch 1/100
649/649 ━━━━━━━━━━━━━━━━━━━━ 0s 493us/step - loss: 0.5425 - val_loss: 6.2301
Epoch 2/100
649/649 ━━━━━━━━━━━━━━━━━━━━ 0s 479us/step - loss: 1.1104 - val_loss: 6.2301
Epoch 3/100
649/649 ━━━━━━━━━━━━━━━━━━━━ 0s 587us/step - loss: 1.0132 - val_loss: 6.2301
Epoch 4/100
649/649 ━━━━━━━━━━━━━━━━━━━━ 0s 470us/step - loss: 0.7913 - val_loss: 6.2301
Epoch 5/100
649/649 ━━━━━━━━━━━━━━━━━━━━ 0s 463us/step - loss: 0.6001 - val_loss: 6.2301
Epoch 6/100
649/649 ━━━━━━━━━━━━━━━━━━━━ 0s 504us/step - loss: 0.9743 - val_loss: 6.2301
Epoch 7/100
649/649 ━━━━━━━━━━━━━━━━━━━━ 0s 453us/step - loss: 0.8245 - val_loss: 6.2301
Epoch 8/100
649/649 ━━━━━━━━━━━━━━━━━━━━ 0s 504us/step - loss: 1.1786 - val_loss: 6.2301
Epoch 9/100
649/649 ━━━━━━━━━━━━━━━━━━━━ 0s 453us/step - loss: 0.9393 - val_loss: 6.2301
Epoch 10/100
649/649 ━━━━━━━━━━━━━━━━━━━━ 0s 450us/step - loss: 0.7220 - val_loss: 6.2301
Epoch 11/100
649/649 ━━━━━━━━━━━━━━━━━━━━ 0s 491us/step - loss: 0.7362 - val_loss: 6.2301
Epoch 12/100
649/64

In [19]:
# Created an evaluation function to track the results of the DNN model
# on all the kinds of data.

def evaluate_dnn(model, X, y_scaled, y_original, label):
    y_pred_scaled = model.predict(X).flatten()
    y_pred = target_scaler.inverse_transform(y_pred_scaled.reshape(-1, 1)).flatten()
    rmse = np.sqrt(mean_squared_error(y_original, y_pred))
    mape = mean_absolute_percentage_error(y_original, y_pred)
    r2 = r2_score(y_original, y_pred)
    print(f"\n {label} Set Performance:")
    print(f"Mean y: {np.mean(y_original):.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"MAPE: {mape:.4f}")
    print(f"R²: {r2:.4f}")
    return rmse, mape, r2

In [20]:
# By using the evaluating function above this is done.

evaluate_dnn(model, X_train_scaled, y_train_scaled, y_train_orig, "Train")
evaluate_dnn(model, X_dev_scaled, y_dev_scaled, y_dev_orig, "Dev")
evaluate_dnn(model, X_test_scaled, y_test_scaled, y_test_orig, "Test")

649/649 ━━━━━━━━━━━━━━━━━━━━ 0s 325us/step

📊 Train Set Performance:
Mean y: 99.9757
RMSE: 0.2598
MAPE: 0.0004
R²: -0.0000
82/82 ━━━━━━━━━━━━━━━━━━━━ 0s 275us/step

📊 Dev Set Performance:
Mean y: 99.9646
RMSE: 0.6484
MAPE: 0.0006
R²: -0.0003
82/82 ━━━━━━━━━━━━━━━━━━━━ 0s 275us/step

📊 Test Set Performance:
Mean y: 99.9775
RMSE: 0.1815
MAPE: 0.0004
R²: -0.0001


(0.18153383910176624, 0.00038275855204272286, -0.00010259928762179626)